In [ ]:
import requests
import datetime

sign = '31ea8aa540a61e824f4d61ca0e23aa6602b599dd53309e27ce2b7d3936cc92f81c9e152dbbf325c3c961fc748c1b019c95b9900b59673a35ff54950760244e30769418079afe1458bab107d09897782e9e661166880043b4e1dce872af48110ad2fadb67fbb831375122a1b06214026958ed837b2f4e8b2a882f14aed03047b368e3ec2c2590c5eb13f57eea11714f9a'
tenantID = '2'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8', '_tenantId': tenantID, '_sign' : sign, 'accept':'*/*'}
pageSize = '50000'

url_api = 'http://ft-api.prod.trimblevi.com:20551/openapi/v2/'
url_fleets = url_api + 'fleets?'
#fleet_names = ['Shaw%20Regional%20-%201333','Shaw%20IPT%20-%201331', 'Shaw%20OTR%20-%202108']
#fleet_names = ['Boise%20Cascade%20-%204241']
fleet_names = ['Quickway%20Dist%20-%201508']
fleetIds=[]
if(len(fleet_names)>0):
    url_fleets = url_fleets + '&fleetName='
    first = True
    for fleet_name in fleet_names :
        url_fleet = url_fleets + fleet_name
        response = requests.get(url_fleet, headers=headers)

        display(url_fleet)

        response_json = response.json()
        fleetIds.append(response_json['data'][0]['fleetId'])

start_date = datetime.datetime(year=2024, month=4, day=19)
start_date_str = start_date.strftime("%Y-%m-%d")

end_date =  datetime.datetime(year=2024, month=4, day=25)
end_date_str = end_date.strftime("%Y-%m-%d")

url_alarms = url_api + 'alarms?alarmTypes=0'

if(len(fleetIds)>0):
    url_alarms = url_alarms + '&includeSubFleet=true&fleetIds='
    first = True
    for fleet_id in fleetIds :
        if(first):
            url_alarms = url_alarms + fleet_id
            first=False
        else:
            url_alarms = url_alarms + '%2C'+ fleet_id

url_alarms = url_alarms + '&endTime='+end_date_str+'T23%3A59%3A59Z&startTime='+start_date_str+'T00%3A00%3A00Z' + '&pageSize=1000'
display(url_alarms)


# Ainda vou melhorar esse trecho
page = 1
output = []
while ((page == 1) or (response.text.find('alarmAddInfo') > 0)):
    url_final = url_alarms + '&page=' + str(page)
    response = requests.get(url_final, headers=headers)
    output.append(response.json())
    page = page + 1

display(output)
